# Introduction to WaterTAP Costing

# DELETE THIS CELL BEFORE MERGING
Outline:
- Brief summary of WaterTAP costing - slides will hopefully cover most of the details
- Just build an RO flowsheet
- Walk through adding the existing costing for each unit model (consider showing snippets of the unit model costing files or documentation with our explanations)
- Walk through how to modify the existing parameters
- How to add system level metrics (LCOW, SEC, etc.)
- Walk through how to go from simulating costing to optimizing it (add objective function for minimizing LCOW)
- Add a try it yourself for optimizing for a different objective (SEC)
- Walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html

## TODO: Mutka should look into adding capital cost components dynamically


This tutorial will demonstrate how to add costing for a reverse osmosis unit model as well as how to change the default parameter values and add custom costing relationships.

## Step 1: Import Modules

In [ ]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.core import UnitModelCostingBlock

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.costing import WaterTAPCosting
from watertap.costing.unit_models.reverse_osmosis import (
    cost_high_pressure_reverse_osmosis,
)
from watertap.costing.unit_models.pump import cost_high_pressure_pump

## Step 2: Create an RO Model

In [ ]:
# Build flowsheet essentials
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Build property model
m.fs.properties = SeawaterParameterBlock()

# Build unit model
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
)

## Step 3: Add RO Costing

### Step 3a: Create the Costing Model

`WaterTAPCosting` contain extensions, methods, and variables and constraints common to all WaterTAP Costing Packages. In other words, this sets up the base cost model, which we will expand upon by adding RO-specific cost terms. This is conventionally defined as `m.fs.costing`, and we'll also establish the base currency for the cost model.

In [ ]:
# Set up WaterTAP Costing
m.fs.costing = WaterTAPCosting()

# Set units of base currency
m.fs.costing.base_currency = pyunits.USD_2020

### Step 3b: Add RO Unit Model Costing Block
Next, add the `UnitModelCostingBlock` for RO, which will set up all the RO-specific variables and constraints related to costing the unit. There are two configuration arguments on `UnitModelCostingBlock` that must be set. The first is `flowsheet_costing_block`, which refers to the instance of WaterTAP costing (usually `m.fs.costing`). The second argument is `costing_method`, which allows the user to choose between different cost assumptions. As shown  below, RO has two costing methods:

<p align="center">
<img src='graphics/ro_costing_doc.png' alt="RO Costing Assumptions" width="1000" height=800>
</p>

In [ ]:
# Create RO costing block
m.fs.RO.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=cost_high_pressure_reverse_osmosis,
)

## Step 4: Display the cost model

In [ ]:
# Print the cost model for RO
m.fs.costing.reverse_osmosis.display()

## Step 5: Modify Existing Parameters

Let's say we want to change the membrane unit cost in the conventional case from 75 USD/m2 to 60 USD/m2. The naming convention for updating these parameters is `m.fs.costing.CostingMethod.Parameter.fix(new_value)`, where `CostingMethod` is the `costing_method` argument without the "cost" prefix.

In [ ]:
# Update RO membrane cost
m.fs.costing.reverse_osmosis.high_pressure_membrane_cost.fix(60)

# Print the cost model for RO
m.fs.costing.reverse_osmosis.display()

## Try It Yourself

Add a `UnitModelCostingBlock` for a high-pressure pump and change the cost from 1.908 USD/W to 1 USD/W given the following information.
<p align="center">
<img src='graphics/pump_costing_doc.png' alt="Pump Costing Assumptions" width="1000" height=800>
</p>

In [ ]:
# Build pump unit model
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Add pump UnitModelCostingBlock
m.fs.pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=cost_high_pressure_pump,
)

# Print the cost model for the pump before updating
m.fs.costing.high_pressure_pump.display()

# Update pump cost parameter
m.fs.costing.high_pressure_pump.cost.fix(1)

# Print the cost model for the pump to verify changes
m.fs.costing.high_pressure_pump.display()

## Step 7: Register Custom Flows

Could walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html, but after some further reflection, I think this is best left for a post-session tutorial. This tutorial functions as a good introduction as it currently stands.